# Create ANR Awards from data.gouv.fr

Creates ANR awards from the French National Research Agency (Agence Nationale de la Recherche) open data. ~34.4K projects.

**Prerequisites:**
- Run `scripts/local/anr_to_s3.py` to download and upload the data first.

**Data sources:**
- ANR_01 (DGDS): https://www.data.gouv.fr/datasets/anr-01-projets-anr-dos-et-dgds-detail-des-projets-et-des-partenaires
- ANR_02 (DGPIE): https://www.data.gouv.fr/datasets/anr-02-projets-anr-dgpie-detail-des-projets-et-des-partenaires

**S3 location:** `s3a://openalex-ingest/awards/anr/anr_projects.parquet`

**ANR funder:**
- funder_id: 4320320883
- ROR: https://ror.org/00rbzpz17
- DOI: 10.13039/501100001665
- display_name: "Agence Nationale de la Recherche"

**Funding programs:**
- DGDS: Directorate for Science (2005-present)
- DGPIE: Major Investment Directorate (2010-present, PIA etc.)
- AAPG: Generic call for projects (annual)
- JCJC: Young researchers (Jeunes Chercheurs et Jeunes Chercheuses)
- BLANC: Open topic research
- ASTRID: Defense research

## Step 1: Create Staging Table from S3

In [ ]:
%sql
-- Create the staging table from S3 parquet
CREATE OR REPLACE TABLE openalex.awards.anr_raw
USING delta
AS
SELECT
    *,
    current_timestamp() as databricks_ingested_at
FROM parquet.`s3a://openalex-ingest/awards/anr/anr_projects.parquet`;

In [ ]:
%sql
-- Check row count (should be ~34.4K)
SELECT COUNT(*) as total_projects FROM openalex.awards.anr_raw;

In [ ]:
%sql
-- Check column names
DESCRIBE openalex.awards.anr_raw;

In [ ]:
%sql
-- Sample the raw data
SELECT * FROM openalex.awards.anr_raw LIMIT 5;

## Step 2: Create ANR Awards Table

In [ ]:
%sql
CREATE OR REPLACE TABLE openalex.awards.anr_awards
USING delta
AS
WITH
-- Get ANR funder from OpenAlex
anr_funder AS (
    SELECT
        funder_id,
        display_name,
        ror_id,
        doi
    FROM openalex.common.funder
    WHERE funder_id = 4320320883  -- Agence Nationale de la Recherche
),

awards_transformed AS (
    SELECT
        -- Generate unique ID using xxhash64 of funder_id:project_id
        abs(xxhash64(CONCAT(f.funder_id, ':', LOWER(g.project_id)))) % 9000000000 as id,

        -- Display name = project title (English preferred, fallback to French)
        g.title as display_name,

        -- Description = abstract
        g.abstract as description,

        -- Funder info
        f.funder_id,
        g.project_id as funder_award_id,

        -- Amount in EUR
        TRY_CAST(g.amount AS DOUBLE) as amount,
        'EUR' as currency,

        -- Funder struct
        struct(
            CONCAT('https://openalex.org/F', f.funder_id) as id,
            f.display_name,
            f.ror_id,
            f.doi
        ) as funder,

        -- Funding type - map common ANR schemes to types
        CASE
            WHEN LOWER(g.funding_program) LIKE '%jcjc%' THEN 'fellowship'
            WHEN LOWER(g.funding_program) LIKE '%jeunes%chercheur%' THEN 'fellowship'
            WHEN LOWER(g.funding_program) LIKE '%chaire%' THEN 'fellowship'
            WHEN LOWER(g.funding_program) LIKE '%postdoc%' THEN 'fellowship'
            WHEN LOWER(g.funding_program) LIKE '%equipex%' THEN 'infrastructure'
            WHEN LOWER(g.funding_program) LIKE '%labex%' THEN 'infrastructure'
            WHEN LOWER(g.funding_program) LIKE '%infrastruct%' THEN 'infrastructure'
            WHEN LOWER(g.funding_program) LIKE '%idex%' THEN 'infrastructure'
            WHEN LOWER(g.funding_program) LIKE '%formation%' THEN 'training'
            WHEN LOWER(g.funding_program) LIKE '%astrid%' THEN 'research'
            WHEN LOWER(g.funding_program) LIKE '%blanc%' THEN 'research'
            WHEN LOWER(g.funding_program) LIKE '%aapg%' THEN 'research'
            ELSE 'grant'
        END as funding_type,

        -- Funder scheme = funding_program
        g.funding_program as funder_scheme,

        -- Provenance
        'anr_opendata' as provenance,

        -- Dates
        TRY_TO_DATE(g.start_date, 'yyyy-MM-dd') as start_date,
        CAST(NULL AS DATE) as end_date,  -- ANR data doesn't include end dates
        COALESCE(
            YEAR(TRY_TO_DATE(g.start_date, 'yyyy-MM-dd')),
            TRY_CAST(g.edition_year AS INT)
        ) as start_year,
        CAST(NULL AS INT) as end_year,

        -- Lead investigator
        CASE
            WHEN g.pi_family_name IS NOT NULL THEN
                struct(
                    g.pi_given_name as given_name,
                    g.pi_family_name as family_name,
                    -- Format ORCID URL if available (ANR data already has ORCID IDs)
                    CASE 
                        WHEN g.pi_orcid IS NOT NULL AND g.pi_orcid != '' THEN
                            CASE 
                                WHEN g.pi_orcid LIKE 'https://orcid.org/%' THEN g.pi_orcid
                                ELSE CONCAT('https://orcid.org/', g.pi_orcid)
                            END
                        ELSE NULL
                    END as orcid,
                    CAST(NULL AS DATE) as role_start,
                    struct(
                        g.institution as name,
                        COALESCE(g.country, 'France') as country,
                        -- Include RNSR code as ID if available
                        CASE 
                            WHEN g.rnsr_code IS NOT NULL THEN
                                array(struct(
                                    g.rnsr_code as id,
                                    'rnsr' as type,
                                    'anr' as asserted_by
                                ))
                            ELSE CAST(NULL AS ARRAY<STRUCT<id:STRING, type:STRING, asserted_by:STRING>>)
                        END as ids
                    ) as affiliation
                )
            ELSE NULL
        END as lead_investigator,

        -- Co-lead and other investigators (not in current data extraction)
        CAST(NULL AS STRUCT<
            given_name:STRING,
            family_name:STRING,
            orcid:STRING,
            role_start:DATE,
            affiliation:STRUCT<name:STRING, country:STRING, ids:ARRAY<STRUCT<id:STRING, type:STRING, asserted_by:STRING>>>
        >) as co_lead_investigator,

        CAST(NULL AS ARRAY<STRUCT<
            given_name:STRING,
            family_name:STRING,
            orcid:STRING,
            role_start:DATE,
            affiliation:STRUCT<name:STRING, country:STRING, ids:ARRAY<STRUCT<id:STRING, type:STRING, asserted_by:STRING>>>
        >>) as investigators,

        -- Landing page URL (ANR award search)
        CONCAT('https://anr.fr/Projet-', g.project_id) as landing_page_url,

        -- No DOI for ANR grants typically
        CAST(NULL AS STRING) as doi,

        -- Works API URL
        concat('https://api.openalex.org/works?filter=awards.id:G', abs(xxhash64(CONCAT(f.funder_id, ':', LOWER(g.project_id)))) % 9000000000) as works_api_url,

        -- Timestamps
        current_timestamp() as created_date,
        current_timestamp() as updated_date

    FROM openalex.awards.anr_raw g
    CROSS JOIN anr_funder f
    WHERE g.project_id IS NOT NULL
      AND TRIM(g.project_id) != ''
)

SELECT * FROM awards_transformed;

In [ ]:
%sql
-- Insert into openalex_awards
INSERT INTO openalex.awards.openalex_awards
SELECT
    id,
    display_name,
    description,
    funder_id,
    funder_award_id,
    amount,
    currency,
    funder,
    funding_type,
    funder_scheme,
    provenance,
    start_date,
    end_date,
    start_year,
    end_year,
    lead_investigator,
    co_lead_investigator,
    investigators,
    landing_page_url,
    doi,
    works_api_url,
    created_date,
    updated_date,
    ARRAY() as funded_outputs,
    0 as funded_outputs_count
FROM openalex.awards.anr_awards;

## Verification Queries

In [ ]:
%sql
-- Check row count (should be ~34.4K)
SELECT COUNT(*) as total_anr_awards FROM openalex.awards.anr_awards;

In [ ]:
%sql
-- Sample the data
SELECT 
    id,
    display_name,
    funder_award_id,
    funder_scheme,
    funding_type,
    amount,
    start_date,
    lead_investigator
FROM openalex.awards.anr_awards 
LIMIT 10;

In [ ]:
%sql
-- Check funder distribution (should all be ANR)
SELECT funder.display_name, COUNT(*) as cnt
FROM openalex.awards.anr_awards
GROUP BY funder.display_name
ORDER BY cnt DESC;

In [ ]:
%sql
-- Check funding_type distribution
SELECT funding_type, COUNT(*) as cnt
FROM openalex.awards.anr_awards
GROUP BY funding_type
ORDER BY cnt DESC;

In [ ]:
%sql
-- Check funder_scheme distribution (top 20 funding programmes)
SELECT funder_scheme, COUNT(*) as cnt
FROM openalex.awards.anr_awards
WHERE funder_scheme IS NOT NULL
GROUP BY funder_scheme
ORDER BY cnt DESC
LIMIT 20;

In [ ]:
%sql
-- Check data completeness
SELECT
    COUNT(*) as total,
    COUNT(display_name) as has_title,
    COUNT(description) as has_abstract,
    COUNT(amount) as has_amount,
    SUM(amount) as total_funding_eur,
    COUNT(start_date) as has_start_date,
    COUNT(lead_investigator) as has_pi,
    SUM(CASE WHEN lead_investigator.orcid IS NOT NULL THEN 1 ELSE 0 END) as has_orcid,
    ROUND(COUNT(lead_investigator) * 100.0 / COUNT(*), 1) as pct_with_pi,
    ROUND(COUNT(description) * 100.0 / COUNT(*), 1) as pct_with_abstract
FROM openalex.awards.anr_awards;

In [ ]:
%sql
-- Check year distribution
SELECT start_year, COUNT(*) as cnt
FROM openalex.awards.anr_awards
WHERE start_year IS NOT NULL
GROUP BY start_year
ORDER BY start_year DESC
LIMIT 25;

In [ ]:
%sql
-- Check lead institutions (top 20)
SELECT 
    lead_investigator.affiliation.name as institution,
    COUNT(*) as cnt
FROM openalex.awards.anr_awards
WHERE lead_investigator.affiliation.name IS NOT NULL
GROUP BY lead_investigator.affiliation.name
ORDER BY cnt DESC
LIMIT 20;

In [ ]:
%sql
-- Check country distribution
SELECT 
    lead_investigator.affiliation.country as country,
    COUNT(*) as cnt
FROM openalex.awards.anr_awards
WHERE lead_investigator.affiliation.country IS NOT NULL
GROUP BY lead_investigator.affiliation.country
ORDER BY cnt DESC
LIMIT 15;